In [1]:
import numpy as np
import math

def SavePts(filename,pts) :
   f=open(filename,"wt")
   for i in range(len(pts)) :
       print ('%d %d %.3f %.3f %.3f'%(pts[i,0],pts[i,1],pts[i,2],pts[i,3],pts[i,4]),file=f)
   f.close()

def SavePtsfile(filename,pts,head) :
   f=open(filename,"wt")
   print ('%d %d %d %d'%(head[0],head[1],head[2],head[3]),file=f)
   for i in range(len(pts)) :
       print ('%d %d %.3f %.3f %.3f'%(pts[i,0],pts[i,1],pts[i,2],pts[i,3],pts[i,4]),file=f)
   f.close()

def position_finale(nbr_couche,nbr_ligne,nbr_colonne,point_actual,point_reference,point_X_ini,distance_critique_int): 
    
    nbr_couche = nbr_couche
    nbr_ligne = nbr_ligne
    nbr_colonne = nbr_colonne
    point_X_fin = point_X_ini
    
    taille_voxel = abs(point_reference[1,2]) - abs(point_reference[0,2])#le voxel est un cube     
    z_1ere_couche = point_reference[0,4]

    for pts in range(len(point_X_ini)):
        if point_X_ini[pts,1] != -1:
            if pts%1000 == 0:
                print('Point: %d' %pts)
            
            couche = round(abs(point_X_ini[pts,4]-z_1ere_couche)/taille_voxel) #numéro de la couche la plus proche du point considéré
            
            j = 0
            n = 0
            det = 0
            while (j<6 or det==0) and (n<6): #minimum 6 points voisins
                    n += 1

                    distance_voisin = []
                    numero = []

                    distance_critique = distance_critique_int * n
                    indice_recherche = distance_critique

                    ##########################################################
                    ###Véfirier si le domaine de recherche déborde l'échantillon
                    ##########################################################
                    if (couche-indice_recherche) < 0:
                        point_pre = 0 ###pts_pre : Premier point
                    else:
                        point_pre = (couche-indice_recherche)*nbr_ligne*nbr_colonne - 1#remplacé par le nombre de ligne et de colonne

                    if (couche+indice_recherche) >= nbr_couche :
                        point_der = nbr_couche*nbr_ligne*nbr_colonne -1 #pts_der : Dernier point
                    else:
                        point_der = (couche+indice_recherche)*nbr_ligne*nbr_colonne -1

                    ##########################################################
                    ###Recherche des pts voisins qui sont bien corrélés
                    ##########################################################  
                    for i in range(int(point_pre), int(point_der)+1):
                    #numéro du premier point et du dernier point dans le domaine de recherche
                        couleur = point_actual[i,1]   
                        #Pour éliminer les points non bien corrélés('0':bleu) 
                        if int(couleur) != -1: 
                            distance_carre = ((point_reference[i,2]-point_X_ini[0,2])**2+(point_reference[i,3]-point_X_ini[0,3])**2+(point_reference[i,4]-point_X_ini[0,4])**2)
                            #Eliminer les pts dont leurs distances supérieures à la distance critique
                            if distance_carre <=(distance_critique*taille_voxel)**2:
                                numero.append(int(point_actual[i,0]))
                                distance_voisin.append(math.sqrt(distance_carre)) 
                                j += 1

                    ##########################################################
                    ###Poids en fonction de distance
                    ##########################################################

                    Lambda = np.zeros([1,len(numero)])
                    for i in range(len(distance_voisin)):
                        Lambda[0,i] = math.exp(-(3*distance_voisin[i]/(2*(distance_critique*taille_voxel)))**2)
                    Lambda = Lambda/np.sum(Lambda)

                    ##########################################################
                    ###Vérifier si matrix est invertible pour calculer F_N,T_N
                    ##########################################################

                    sum3 = np.matrix(np.zeros(shape=(3,3))) ###enregistrer la somme de <X⊗X>
                    sum_X = np.matrix(np.zeros(shape=(3,1)))###enregistrer la somme de X

                    for i in range(len(numero)):       
                        sum3 += Lambda[0,i]*(np.outer(point_reference[numero[i],2:],point_reference[numero[i],2:]))
                        sum_X += Lambda[0,i]*(point_reference[numero[i],2:].reshape(3,1))

                    t3 = sum3 #<X⊗X>
                    t4 = np.outer(sum_X,sum_X) #<X>⊗<X>

                    det = np.linalg.det(t3-t4)

            ##########################################################
            ###Calculer le gradient global de déplacement F
            ##########################################################
            ###F = [<x⊗X>-<x>⊗<X>]·[<X⊗X>-<X>⊗<X>]^(-1) avec x: position finale et X: position initiale

            if n<6:
                sum1 = np.matrix(np.zeros(shape=(3,3))) ###enregistrer la somme de x⊗X
                #um3 = np.matrix(np.zeros(shape=(3,3))) ###enregistrer la somme de <X⊗X>
                sum_x = np.matrix(np.zeros(shape=(3,1)))###enregistrer la somme de x
                #um_X = np.matrix(np.zeros(shape=(3,1)))###enregistrer la somme de X
                mat_unit = np.eye(3)                    ### matrice identifiée

                for i in range(len(numero)):
                    sum1 +=Lambda[0,i]*(np.outer(point_actual[numero[i],2:],point_reference[numero[i],2:]))
                    sum_x += Lambda[0,i]*(point_actual[numero[i],2:].reshape(3,1))            
                    #um3 += Lambda[0,i]*(np.outer(point_reference[numero[i],2:],point_reference[numero[i],2:]))
                    #um_X += Lambda[0,i]*(point_reference[numero[i],2:].reshape(3,1)) 

                t1 = sum1 #<x⊗X>
                t2 = np.outer(sum_x,sum_X)#<x>⊗<X>
                #3 = sum3 #<X⊗X>
                #4 = np.outer(sum_X,sum_X) #<X>⊗<X>
                mat_inverse = np.linalg.inv(t3-t4)
                F = np.dot((t1-t2),mat_inverse)
                F_fit = F-mat_unit
                ###T = <x>-F·<X>
                T = sum_x - np.dot(F,sum_X)

                ##########################################################
                ###Position finale avec le pts plus proche
                ##########################################################
                ### Interpolation moyenne (T+F(X))

                point_X_fin[pts,2:] = np.transpose(T + np.dot(F,point_X_ini[pts,2:].reshape(3,1)))
            else:
                point_X_fin[pts,1] = -1
    return point_X_fin       

In [2]:
point_actual = np.array((np.loadtxt('039_LA01_step11_spiral_00_32x32x19.txt')))#040_LA01_step12_spiral_00_32x32x19.txt
point_reference = np.array((np.loadtxt('028_LA01_step00_spiral_00_32x32x19.txt')))
#point_X_ini = np.array([[1, 0, 1049.633,-969.632,-1418.055]]) 
point_X_ini = np.array((np.loadtxt('Ref_Grille_0.txt')))

point_X_fin = position_finale(19,32,32,point_actual,point_reference,point_X_ini,2)

print('Save file: 0')
SavePts('Step11_Grille_0.txt', point_X_fin)
SavePtsfile('Step11_Grille_0.pts', point_X_fin, [32*32*19, 19, 32, 32])

#######################################################################################

point_actual = np.array((np.loadtxt('039_LA01_step11_spiral_01_32x32x19.txt')))
point_reference = np.array((np.loadtxt('028_LA01_step00_spiral_01_32x32x19.txt')))
#point_X_ini = np.array([[1, 0, 1049.633,-969.632,-1418.055]]) 
point_X_ini = np.array((np.loadtxt('Ref_Grille_1.txt')))

point_X_fin = position_finale(19,32,32,point_actual,point_reference,point_X_ini,2)

print('Save file: 1')
SavePts('Step11_Grille_1.txt', point_X_fin)
SavePtsfile('Step11_Grille_1.pts', point_X_fin, [32*32*17, 17, 32, 32])

#######################################################################################

point_actual = np.array((np.loadtxt('039_LA01_step11_spiral_02_32x32x19.txt')))
point_reference = np.array((np.loadtxt('028_LA01_step00_spiral_02_32x32x19.txt')))
#point_X_ini = np.array([[1, 0, 1049.633,-969.632,-1418.055]]) 
point_X_ini = np.array((np.loadtxt('Ref_Grille_2.txt')))

point_X_fin = position_finale(19,32,32,point_actual,point_reference,point_X_ini,2)

print('Save file: 2')
SavePts('Step11_Grille_2.txt', point_X_fin)
SavePtsfile('Step11_Grille_2.pts', point_X_fin, [32*32*17, 17, 32, 32])


#######################################################################################

point_actual = np.array((np.loadtxt('039_LA01_step11_spiral_03_32x32x19.txt')))
point_reference = np.array((np.loadtxt('028_LA01_step00_spiral_03_32x32x19.txt')))
#point_X_ini = np.array([[1, 0, 1049.633,-969.632,-1418.055]]) 
point_X_ini = np.array((np.loadtxt('Ref_Grille_3.txt')))

point_X_fin = position_finale(19,32,32,point_actual,point_reference,point_X_ini,2)

print('Save file: 3')
SavePts('Step11_Grille_3.txt', point_X_fin)
SavePtsfile('Step11_Grille_3.pts', point_X_fin, [32*32*17, 17, 32, 32])

Point: 0
Point: 1000
Point: 2000
Point: 3000
Point: 4000
Point: 5000
Point: 6000
Point: 7000
Point: 8000
Point: 9000
Point: 10000
Point: 11000
Point: 12000
Point: 13000
Point: 14000
Point: 15000
Point: 16000
Point: 17000
Point: 18000
Point: 19000
Save file: 0
Point: 0
Point: 1000
Point: 2000
Point: 3000
Point: 4000
Point: 5000
Point: 6000
Point: 7000
Point: 8000
Point: 9000
Point: 10000
Point: 11000
Point: 12000
Point: 13000
Point: 14000
Point: 15000
Point: 16000
Point: 17000
Save file: 1
Point: 0
Point: 1000
Point: 2000
Point: 3000
Point: 4000
Point: 5000
Point: 6000
Point: 7000
Point: 8000
Point: 9000
Point: 10000
Point: 11000
Point: 12000
Point: 13000
Point: 14000
Point: 15000
Point: 16000
Point: 17000
Save file: 2
Point: 0
Point: 1000
Point: 2000
Point: 3000
Point: 4000
Point: 5000
Point: 6000
Point: 7000
Point: 8000
Point: 9000
Point: 10000
Point: 11000
Point: 12000
Point: 13000
Point: 14000
Point: 15000
Point: 16000
Point: 17000
Save file: 3
